<a href="https://colab.research.google.com/github/simulate111/Deep-Learning-in-Human-Language-Technology/blob/main/course_project_rezac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning in Human Language Technology Project (Template)

- Student(s) Name(s): Mohammadreza Akhtari
- Date: October 2024
- Chosen Corpus:
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus:
- Paper(s) and other published materials related to the corpus:
- Random baseline performance and expected performance for recent machine learned models:

---

## 1. Setup

In [1]:
# Your code to install and import libraries etc. here
!pip install transformers datasets evaluate
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict
import evaluate
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00


---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [2]:
# Your code to download the corpus here
#First, training and evaluation on english dataset is requested
dataset_en = load_dataset("mteb/amazon_reviews_multi", "en", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

amazon_reviews_multi.py:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/28.3M [00:00<?, ?B/s]

en/validation/0000.parquet:   0%|          | 0.00/713k [00:00<?, ?B/s]

en/test/0000.parquet:   0%|          | 0.00/711k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [3]:
print(dataset_en)
#Dataset consists of 1200000 data with the 30000 data for validation and 30000 for test dataset.
#Data fetures are id, text, label, and label text.
#However, considering the only English or any other language database, there is 200000 of data with 5000 for validation and 5000 for test datasets.

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [4]:
# HEre, you could find some more information about the dataset provided by the provider.
print(dataset_en['train'].info)

DatasetInfo(description='We provide an Amazon product reviews dataset for multilingual text classification. The dataset contains reviews in English, Japanese, German, French, Chinese and Spanish, collected between November 1, 2015 and November 1, 2019. Each record in the dataset contains the review text, the review title, the star rating, an anonymized reviewer ID, an anonymized product ID and the coarse-grained product category (e.g. ‘books’, ‘appliances’, etc.) The corpus is balanced across stars, so each star rating constitutes 20% of the reviews in each language.\nFor each language, there are 200,000, 5,000 and 5,000 reviews in the training, development and test sets respectively. The maximum number of reviews per reviewer is 20 and the maximum number of reviews per product is 20. All reviews are truncated after 2,000 characters, and all reviews are at least 20 characters long.\nNote that the language of a review does not necessarily match the language of its marketplace (e.g. revi

### 2.2. Sampling and preprocessing

In [5]:
# Your code for any necessary sampling and preprocessing here
#Downsizing the dataset to make the computation faster and possible
#The origin dataset has 1,200,000 training data and 30,000 for each of test and validation set. The traiing set is dowsized to 12,000 data and 3,000 for validation and test datasets.
#Data is shuffled to make sampling reasonable and with good distribution of different data class.
train_dataset_en = dataset_en['train'].shuffle().select(range(int(len(dataset_en['train']) * 0.1)))
val_dataset_en = dataset_en['validation'].shuffle().select(range(int(len(dataset_en['validation']) * 0.5)))
test_dataset_en = dataset_en['test'].shuffle().select(range(int(len(dataset_en['test']) * 1)))

# Now, a new dataset for further analysis is made here from the dowscaled data.
downsampled_dataset_en = DatasetDict({'train': train_dataset_en, 'validation': val_dataset_en, 'test': test_dataset_en})
print(downsampled_dataset_en)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [6]:
# Label distribution to explore the evenly distribution of labels
for split in downsampled_dataset_en.keys():
    print(f"{split} labels:")
    labels = downsampled_dataset_en[split]['label']
    unique_labels, counts = np.unique(labels, return_counts=True)
    for label, count in zip(unique_labels, counts):
        print(f"'{label}': {count} samples")
    print()

train labels:
'0': 3986 samples
'1': 4039 samples
'2': 3953 samples
'3': 3984 samples
'4': 4038 samples

validation labels:
'0': 514 samples
'1': 498 samples
'2': 497 samples
'3': 493 samples
'4': 498 samples

test labels:
'0': 1000 samples
'1': 1000 samples
'2': 1000 samples
'3': 1000 samples
'4': 1000 samples



In [7]:
# Your code for any necessary sampling and preprocessing here
#Use bert-base-cased model as was used also during the exercises.
model_name = "bert-base-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Define a simple function that applies the tokenizer
# maximum length of BERT models is 512 due to the position embeddings
def tokenize(example):
    return tokenizer(
        example["text"],
        max_length=512,
        truncation=True)
tokenized_datasets_en = downsampled_dataset_en.map(tokenize)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

---

## 3. Machine learning model

### 3.1. Model training

In [9]:
# Your code to train the transformer based model on the training set and evaluate the performance on the validation set here
#Taking advantage of course exercises
#There are few languages in the datset. Therefore, we use multilingual model here.
model_name = "bert-base-multilingual-cased"
# Initialize the model
#We have 5 labels here as we want to give 1 to 5 stars or label 0 to 4.
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Set training arguments lr=1e-5
training_argse5_500 = transformers.TrainingArguments(
    output_dir="checkpoints",
    eval_strategy="steps",
    logging_strategy="no",
    load_best_model_at_end=True,
    eval_steps=100,
    learning_rate=0.00001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=500,
    report_to="none")

In [11]:
# Load the accuracy metric
accuracy = evaluate.load("accuracy")

# Define the compute_accuracy function
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = outputs.argmax(axis=-1)  # Pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

# Collator
data_collator = DataCollatorWithPadding(tokenizer)

# Argument gives the number of steps of patience before early stopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5)

In [12]:
# Print a sample of test labels to see that they are not ordered
print(downsampled_dataset_en["test"]["label"][:100])

[3, 3, 4, 1, 0, 4, 2, 0, 1, 1, 3, 4, 1, 4, 2, 2, 3, 2, 1, 3, 1, 2, 2, 3, 4, 4, 0, 4, 3, 3, 4, 2, 1, 2, 2, 0, 2, 1, 4, 4, 2, 3, 0, 1, 1, 0, 3, 0, 4, 1, 0, 2, 0, 2, 1, 1, 2, 4, 1, 1, 3, 2, 3, 4, 0, 4, 2, 4, 2, 2, 3, 3, 0, 1, 2, 4, 4, 3, 0, 1, 0, 0, 3, 0, 1, 0, 2, 0, 0, 2, 2, 1, 1, 0, 2, 4, 3, 3, 3, 4]


In [13]:
# Initialize the Trainer
trainer_en_e5_500 = transformers.Trainer(
    model=model,
    args=training_argse5_500,
    train_dataset=tokenized_datasets_en["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping])

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer_en_e5_500.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,1.618051,0.213200
200,No log,1.558389,0.294000
300,No log,1.536767,0.279600
400,No log,1.512472,0.318800
500,No log,1.500417,0.335600


TrainOutput(global_step=500, training_loss=1.5679580078125, metrics={'train_runtime': 278.6365, 'train_samples_per_second': 14.356, 'train_steps_per_second': 1.794, 'total_flos': 254468133530496.0, 'train_loss': 1.5679580078125, 'epoch': 0.2})

### 3.2 Hyperparameter optimization

In [15]:
# Set training arguments lr=1e-5 and step 2000 instead of 500
training_argse5_1000 = transformers.TrainingArguments(
    output_dir="checkpoints",
    eval_strategy="steps",
    logging_strategy="no",
    load_best_model_at_end=True,
    eval_steps=100,
    learning_rate=0.00001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=1000,
    report_to="none")

In [16]:
# Initialize the Trainer
trainer_en_e5_1000 = transformers.Trainer(
    model=model,
    args=training_argse5_1000,
    train_dataset=tokenized_datasets_en["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping])

max_steps is given, it will override any value given in num_train_epochs


In [17]:
trainer_en_e5_1000.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,1.477194,0.328400
200,No log,1.447815,0.361200
300,No log,1.413728,0.363200
400,No log,1.394341,0.408800
500,No log,1.295086,0.432800
600,No log,1.271805,0.448400
700,No log,1.258501,0.456000
800,No log,1.234097,0.470800
900,No log,1.227329,0.466000
1000,No log,1.217456,0.467600


TrainOutput(global_step=1000, training_loss=1.32520361328125, metrics={'train_runtime': 618.0425, 'train_samples_per_second': 12.944, 'train_steps_per_second': 1.618, 'total_flos': 504179584391760.0, 'train_loss': 1.32520361328125, 'epoch': 0.4})

In [18]:
# Set training arguments lr=1e-5 and step 2000 instead of 500
training_argse5_2000 = transformers.TrainingArguments(
    output_dir="checkpoints",
    eval_strategy="steps",
    logging_strategy="no",
    load_best_model_at_end=True,
    eval_steps=100,
    learning_rate=0.00001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=2000,
    report_to="none")

In [19]:
# Initialize the Trainer
trainer_en_e5_2000 = transformers.Trainer(
    model=model,
    args=training_argse5_2000,
    train_dataset=tokenized_datasets_en["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping])

max_steps is given, it will override any value given in num_train_epochs


In [20]:
trainer_en_e5_2000.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,1.290148,0.457200
200,No log,1.351036,0.462800
300,No log,1.414121,0.435600
400,No log,1.337431,0.467200
500,No log,1.292895,0.464400
600,No log,1.296996,0.470400
700,No log,1.293203,0.465200
800,No log,1.237732,0.481200
900,No log,1.196526,0.498000
1000,No log,1.170057,0.498400


TrainOutput(global_step=2000, training_loss=1.0820218505859376, metrics={'train_runtime': 1253.6194, 'train_samples_per_second': 12.763, 'train_steps_per_second': 1.595, 'total_flos': 1014900121356336.0, 'train_loss': 1.0820218505859376, 'epoch': 0.8})

In [21]:
# examine different learning rate

# Set training arguments lr=1e-3
training_argse3_500 = transformers.TrainingArguments(
    output_dir="checkpoints",
    eval_strategy="steps",
    logging_strategy="no",
    load_best_model_at_end=True,
    eval_steps=100,
    learning_rate=0.001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=500,
    report_to="none")
# Set training arguments lr=1e-1
training_argse1_500 = transformers.TrainingArguments(
    output_dir="checkpoints",
    eval_strategy="steps",
    logging_strategy="no",
    load_best_model_at_end=True,
    eval_steps=100,
    learning_rate=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=500,
    report_to="none")

In [22]:
# Initialize the Trainer
trainer_en_e3_500 = transformers.Trainer(
    model=model,
    args=training_argse3_500,
    train_dataset=tokenized_datasets_en["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping])

max_steps is given, it will override any value given in num_train_epochs


In [23]:
# Initialize the Trainer
trainer_en_e1_500 = transformers.Trainer(
    model=model,
    args=training_argse1_500,
    train_dataset=tokenized_datasets_en["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping])

max_steps is given, it will override any value given in num_train_epochs


In [24]:
trainer_en_e3_500.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,1.674876,0.198800
200,No log,1.697025,0.199200
300,No log,1.684598,0.197200
400,No log,1.621977,0.205600
500,No log,1.628960,0.199200


TrainOutput(global_step=500, training_loss=1.6840643310546874, metrics={'train_runtime': 270.5892, 'train_samples_per_second': 14.783, 'train_steps_per_second': 1.848, 'total_flos': 254468133530496.0, 'train_loss': 1.6840643310546874, 'epoch': 0.2})

In [25]:
trainer_en_e1_500.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,10.464881,0.198800
200,No log,6.875025,0.197200
300,No log,4.379682,0.197200
400,No log,7.553667,0.199200
500,No log,1.878205,0.199200


TrainOutput(global_step=500, training_loss=12.805171875, metrics={'train_runtime': 305.2129, 'train_samples_per_second': 13.106, 'train_steps_per_second': 1.638, 'total_flos': 254468133530496.0, 'train_loss': 12.805171875, 'epoch': 0.2})

In [26]:
#learning_rate=0.0000001
training_argse7_500 = transformers.TrainingArguments(
    output_dir="checkpoints",
    eval_strategy="steps",
    logging_strategy="no",
    load_best_model_at_end=True,
    eval_steps=100,
    learning_rate=0.0000001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=500,
    report_to="none")

In [27]:
# Initialize the Trainer
trainer_en_e7_500 = transformers.Trainer(
    model=model,
    args=training_argse7_500,
    train_dataset=tokenized_datasets_en["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping])

max_steps is given, it will override any value given in num_train_epochs


In [28]:
trainer_en_e7_500.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,1.877215,0.199200
200,No log,1.876759,0.199200
300,No log,1.876075,0.199200
400,No log,1.875750,0.199200
500,No log,1.875739,0.199200


TrainOutput(global_step=500, training_loss=4.093751953125, metrics={'train_runtime': 282.3878, 'train_samples_per_second': 14.165, 'train_steps_per_second': 1.771, 'total_flos': 254468133530496.0, 'train_loss': 4.093751953125, 'epoch': 0.2})

### 3.3. Evaluation on test set

In [29]:
# Your code to evaluate the final model on the test set here
# Evaluation on test dataset
test_results = trainer_en_e5_500.evaluate(eval_dataset=tokenized_datasets_en["test"])
print(f"Accuracy: {test_results['eval_accuracy']}")

Accuracy: 0.2


In [30]:
test_resultse1 = trainer_en_e1_500.evaluate(eval_dataset=tokenized_datasets_en["test"])
print(f"Accuracye1: {test_resultse1['eval_accuracy']}")

Accuracye1: 0.2


In [31]:
test_resultse3 = trainer_en_e3_500.evaluate(eval_dataset=tokenized_datasets_en["test"])
print(f"Accuracye1: {test_resultse3['eval_accuracy']}")

Accuracye1: 0.2


In [32]:
test_resultse7 = trainer_en_e7_500.evaluate(eval_dataset=tokenized_datasets_en["test"])
print(f"Accuracye1: {test_resultse7['eval_accuracy']}")

Accuracye1: 0.2


In [1]:
test_resultse5_2000 = trainer_en_e5_2000.evaluate(eval_dataset=tokenized_datasets_en["test"])
print(f"Accuracye1: {test_resultse5_2000['eval_accuracy']}")

NameError: name 'trainer_en_e5_2000' is not defined

### 3.4. Cross-lingual experiments

In [ ]:
# Your code to train and evaluate the cross-lingual model
# Try with German language and dataset
dataset_de = load_dataset("mteb/amazon_reviews_multi", "de")

# Tokenize the dataset
tokenized_datasets_de = dataset_de.map(tokenize, batched=True)
trainer_de = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_de["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=DataCollatorWithPadding(tokenizer),
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)])

trainer_de.train()

In [ ]:
zero_shot_results = trainer_de.evaluate(eval_dataset=tokenized_datasets_en["test"])
print("Accuracy of trained on German but evaluated on English:",zero_shot_results['eval_accuracy'])

In [ ]:
#How about japanese which is really different from european alphabet and languages
dataset_ja = load_dataset("mteb/amazon_reviews_multi", "ja")
trainer_ja = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_ja["train"],
    eval_dataset=tokenized_datasets_en["validation"],
    compute_metrics=compute_accuracy,
    data_collator=DataCollatorWithPadding(tokenizer),
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)])

trainer_ja.train()

In [ ]:
#Evaluation
zero_shot_results_ja = trainer_ja.evaluate(eval_dataset=tokenized_datasets_en["test"])
print("Accuracy of trained on Japanese but evaluated on English:",zero_shot_results_ja['eval_accuracy'])

---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to random baseline / expected performance / state of the art

(Compare your results with the random and state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. and 5.2. Model and Data selection

(Briefly describe which model was used and why. Also, describe how the test data was downsampled, include relevant code.)

### 5.3. Prompt design

(Include your final prompt here. Also, explain here all prompt engineering insights you learned during the project.)

### 5.4. Generate

In [ ]:
# Your code to run the generative model and extract predictions from the model output.

### 5.5. Evaluation and results

(Briefly summarize your results)

### 5.6 Error analysis (group projects only)

(Present the error analysis results here)